In [44]:
import os
import sys

current_dir = os.path.abspath('') 
root_dir = os.path.join(current_dir, "../")
sys.path.insert(0, root_dir)

print(sys.path)
import pandas as pd
import numpy as np
import string
from sklearn import model_selection, metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import utils, config 
from engine import engine_lr, engine_nn, engine_bert
from processing import processing, emb_processing
from sklearn.preprocessing import LabelEncoder

import tensorflow.keras as keras
import tensorflow as tf
from keras.layers import LSTM, Dense, Flatten

import pickle

%load_ext autoreload
%autoreload 2

['/home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/../', '/home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/../', '/home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/../', '', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python36.zip', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/lib-dynload', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages', '/home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/IPython/extensions', '/home/cedric/.ipython']
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [119]:
# Load data 
data_train = utils.load_data(config.TRAIN_PATH)
data_val = utils.load_data(config.VAL_PATH)

data_initial = pd.concat([data_train, data_val])

# Load tweets
tweet_data = utils.load_tweet_data()
tweet_data = tweet_data[tweet_data.sentiment.isin(["sadness","happiness","love","hate","anger", "surprise", "neutral"])]



# Merge two dataframes sources
all_data = pd.concat([data_initial, tweet_data])

sentiment_map = {
        "surprise": "joy",
        "happiness": "joy",
        "love": "love",
        "sadness": "sadness",
        "hate": "anger",
        "anger": "anger",
        "joy": "joy",
        "love": "love",
        "fear": "fear"
        
    }
all_data["sentiment"] = list(map(sentiment_map.get, all_data["sentiment"].values))
data_merge = all_data[all_data.sentiment.isin(["sadness", "anger", "love", "joy", "merge"])]

In [49]:
data_merge[config.TARGET_COL].unique()

array(['sadness', 'anger', 'love', 'joy'], dtype=object)

In [50]:
# Devide data set between train and test
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    data_merge.drop(config.TARGET_COL, axis=1),
    data_merge[config.TARGET_COL],
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=data_merge[config.TARGET_COL].values
)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

y_val_encoded = label_encoder.transform(y_test)

In [51]:
data_merge[config.TARGET_COL].value_counts()

joy        14115
sadness    10381
love        5324
anger       3867
Name: sentiment, dtype: int64

In [52]:
label_encoder.inverse_transform([0,1,2,3])

array(['anger', 'joy', 'love', 'sadness'], dtype=object)

In [23]:
class_weight = {
    0: 14115/3867,
    1: 14115/14115,
    2: 14115/5324,
    3: 14115/10381
}

# Logistic Regression with TF-IDF

In [126]:
data_processeur = processing.data_preprocess() # defini la suite des preprocessing
processed_X_train = data_processeur.fit_transform(X_train)

_, lsa_pip, _ = data_processeur.transformers_[0]
_, countVec = lsa_pip.steps[1]

print(f"nombre de mots du dict : {len(countVec.vocabulary_)}")

nombre de mots du dict : 2583


In [127]:
countVec.vocabulary_ 
#print(data_processeur)

{'love': 1351,
 'feel': 790,
 'hot': 1081,
 'beauti': 190,
 'sexi': 1974,
 'know': 1247,
 'holi': 1059,
 'spirit': 2111,
 'work': 2528,
 'plus': 1697,
 'divin': 609,
 'god': 932,
 'way': 2472,
 'like': 1309,
 'massag': 1389,
 'reassur': 1817,
 'frustrat': 876,
 'tire': 2284,
 'today': 2287,
 'amaz': 78,
 'great': 961,
 'photographi': 1673,
 'passion': 1638,
 'combin': 410,
 'smug': 2067,
 'happi': 1001,
 'help': 1039,
 'proud': 1761,
 'grate': 958,
 'america': 79,
 'honor': 1070,
 'part': 1631,
 'year': 2560,
 'oh': 1567,
 'thunderstorm': 2272,
 'boo': 251,
 'alway': 76,
 'make': 1373,
 'power': 1718,
 'go': 930,
 'id': 1105,
 'terribl': 2235,
 'nightmar': 1532,
 'littl': 1321,
 'disturb': 607,
 'come': 411,
 'realize': 1814,
 'often': 1566,
 'blame': 228,
 'thing': 2254,
 'utter': 2408,
 'outsid': 1606,
 'control': 451,
 'good': 939,
 'music': 1484,
 'lyric': 1362,
 'seem': 1952,
 'human': 1091,
 'lose': 1344,
 'abil': 4,
 'understand': 2368,
 'what': 2492,
 'bad': 167,
 'want': 2462,

In [128]:

label_encoder_lr = LabelEncoder()
y_train_encoded = label_encoder_lr.fit_transform(y_train)

lr_model = engine_lr.train_fn(processed_X_train, y_train_encoded)

preds_proba = lr_model.predict_proba(processed_X_train)
preds = lr_model.predict(processed_X_train)
acc = accuracy_score(y_train_encoded, preds)
print(f"====> Acc : {acc}")
print(confusion_matrix(y_train_encoded,preds))

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Batch computation too fast (0.0580s.) Setting batch_size=2.
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done  24 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Done  38 tasks      | elapsed:    3.2s
[Parallel(n_jobs=2)]: Done  52 tasks      | elapsed:    4.3s


{'C': 10, 'penalty': 'l2', 'solver': 'saga'}
====> Acc : 0.600632911392405
[[309  26  12 108]
 [ 63 148  19 102]
 [ 38  31 158  52]
 [123  28  29 334]]


[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:    5.6s finished


# NN + embedding models

In [58]:
data_merge = data_merge.sample(frac=1)

#x_clean = preprocess_func.transform(X)
x_clean = emb_processing.clean_text(X_train)
x_input, tokenizer = emb_processing.tokenize_text(x_clean, istraining=True)

x_test_clean = emb_processing.clean_text(X_test)
x_test_input, _ = emb_processing.tokenize_text(x_test_clean, tokenizer=tokenizer)


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train)
y_labels = keras.utils.to_categorical(y_encoded)

y_test_encoded = label_encoder.transform(y_test)
y_test_labels = keras.utils.to_categorical(y_test_encoded)

In [59]:
x_clean.head(20)

25607                    oh excellent song hello my friend
5539     i be feel that i will never be disturb by the ...
7251     god i hate scary movie they be not fun at all ...
13156    he be not there today my free day from when th...
23780    get an awesome hair cut today it look so hotha...
18738                                          i suck dick
903      i be ask to lead a prayer meeting i feel a sol...
32610    love the album guy and can not wait for the of...
2749     i do not feel like a child and yet i could see...
4374     i often feel the need to defend just about any...
7672           g night world vacation have to end some day
9718             just get home from a long day at asda joy
17       so sleepy again and it be not even that late i...
10106    hang with my bff today ya ya our old friend be...
12814    i go i be amaze at what i have and i begin to ...
10775    i sit in one of the rocking chair and let my h...
7428        i still feel somewhat disappointed with myse

In [28]:
_, _, word_emb = emb_processing.read_embedded_vecs(config.EMBEDDING_FILE_PATH)

In [61]:
# Search for reccurent words
limite_apparition = 3

from collections import Counter
results = Counter()
x_clean.str.split().apply(results.update)
results = results.most_common()
reccurent_words = [tuple[0] for tuple in results if (tuple[1]>= limite_apparition) and (tuple[0] not in word_emb.keys())]
print([tuple for tuple in results if tuple[0] not in word_emb.keys()])

[('tweeps', 11), ('yayy', 10), ('hehehe', 9), ('heyy', 9), ('kewl', 8), ('awh', 8), ('nkotb', 8), ('mothersday', 8), ('sux', 8), ('bhaha', 8), ('yeahh', 7), ('^^', 7), ('gunna', 7), ('-_-', 7), ('tweeple', 7), ('twitterverse', 6), ('everyones', 6), ('rofl', 6), ('-haha', 6), ('retweet', 6), ('enthral', 6), ('twitterland', 6), ('o_o', 5), ('ughh', 5), ('omfg', 5), ('tbh', 5), ('todayy', 5), ('dayy', 5), ('unfollowed', 5), ('yaay', 5), ('convo', 5), ('hahaha', 5), ('ahaha', 5), ('omj', 5), ('becuz', 5), ('twitterer', 5), ('xmen', 5), ('-d', 5), ('ofcourse', 4), ('dangit', 4), ('tomorow', 4), ('xoxox', 4), ('hols', 4), ('it-', 4), ('mehaha', 4), ('teehee', 4), ('followfriday', 4), ('otalia', 4), ('hapy', 4), ('bleh', 4), ('awesomness', 4), ('tmrw', 4), ('unfollow', 4), ('knoww', 4), ('dayhaha', 4), ('muah', 4), ('toohaha', 4), ('yey', 4), ('hairdresser~~', 4), ('wooho', 4), ('twitts', 4), ('sucky', 4), ('twitterville', 4), ('youhaha', 4), ('probs', 4), ('ftsk', 4), ('lolz', 4), ('wld', 3)

In [62]:
len(reccurent_words)

114

In [90]:
x_input.shape

(26949, 40)

In [63]:
# load embedding dict and compute embedding matrix
emb_matrix = emb_processing.get_embedding_matrix(word_emb, tokenizer, reccurent_words=reccurent_words)

In [112]:
# Fix seed 
np.random.seed(1)
tf.random.set_seed(2)

n_classes = 4
epochs = 6

model_1 = engine_nn.FeedForwardNn(embedding_matrix=emb_matrix, n_classes=n_classes)
model_2 = engine_nn.LstmNetwork(embedding_matrix=emb_matrix, n_classes=n_classes)
model_3 = engine_nn.MixNetwork(embedding_matrix=emb_matrix, n_classes=n_classes)

opt = keras.optimizers.Adam(learning_rate=0.005)
model_1.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=[utils.mcc_metric, "acc"])
model_2.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=[utils.mcc_metric, "acc"])
model_3.compile(loss='categorical_crossentropy',optimizer="Adam", metrics=[utils.mcc_metric, "acc"])

#model_1.fit(x=x_input, y=y_labels, verbose=2, epochs=epochs, class_weight=class_weight)
#model_2.fit(x=x_input, y=y_labels, verbose=2, epochs=epochs, class_weight=class_weight)
model_3.fit(x=x_input, y=y_labels, verbose=2, validation_data=(x_test_input, y_test_labels), epochs=epochs, class_weight=class_weight)

Epoch 1/6
843/843 - 27s - loss: 1.9658 - mcc_metric: 0.1809 - acc: 0.3854 - val_loss: 1.1428 - val_mcc_metric: 0.2879 - val_acc: 0.4659
Epoch 2/6
843/843 - 31s - loss: 1.5915 - mcc_metric: 0.4241 - acc: 0.5683 - val_loss: 0.8991 - val_mcc_metric: 0.4882 - val_acc: 0.6159
Epoch 3/6
843/843 - 30s - loss: 1.3808 - mcc_metric: 0.5002 - acc: 0.6256 - val_loss: 0.8695 - val_mcc_metric: 0.4962 - val_acc: 0.6221
Epoch 4/6
843/843 - 31s - loss: 1.2649 - mcc_metric: 0.5324 - acc: 0.6494 - val_loss: 0.8723 - val_mcc_metric: 0.4899 - val_acc: 0.6175
Epoch 5/6
843/843 - 30s - loss: 1.1793 - mcc_metric: 0.5583 - acc: 0.6688 - val_loss: 0.7731 - val_mcc_metric: 0.5448 - val_acc: 0.6585
Epoch 6/6
843/843 - 31s - loss: 1.1061 - mcc_metric: 0.5779 - acc: 0.6831 - val_loss: 0.7356 - val_mcc_metric: 0.6082 - val_acc: 0.7060


# Inspect results on train 

In [110]:
predictions = model_3.predict(x_input)

winning_class = np.apply_along_axis(lambda x: np.argmax(x), 1, predictions)
winning_class_pred = np.apply_along_axis(lambda x: np.max(x), 1, predictions)
winning_class_name = label_encoder.inverse_transform(winning_class)

train_copie = X_train.copy()
train_copie["clean"] = x_clean.values
train_copie["value"] = y_train.values
train_copie["pred"] = winning_class_name
train_copie["pred_prop"] = winning_class_pred

train_copie["match"] = train_copie.apply(lambda x: int(x.value==x.pred), axis=1)
train_copie.head()

,text,clean,value,pred,pred_prop,match
25607,oh excellent song hello my friend,oh excellent song hello my friend,joy,joy,0.625523,1
5539,im feeling that i will never being disturb by ...,i be feel that i will never be disturb by the ...,love,love,0.542120,1
7251,god i hate scary movies they are not fun at al...,god i hate scary movie they be not fun at all ...,anger,anger,0.935995,1
13156,he wasn t there today my free day from when th...,he be not there today my free day from when th...,sadness,sadness,0.654117,1
23780,got an awesome hair cut today it looks so hoth...,get an awesome hair cut today it look so hotha...,joy,joy,0.604038,1


In [111]:
print(train_copie[train_copie.match==1].value.value_counts())
print(train_copie[train_copie.match==0].value.value_counts())
pd.crosstab(train_copie.value, train_copie.pred)

joy        8266
sadness    7333
love       3586
anger      2848
Name: value, dtype: int64
joy        3026
sadness     972
love        673
anger       245
Name: value, dtype: int64


pred,anger,joy,love,sadness
value,,,,
anger,2848,30,0,215
joy,106,8266,1574,1346
love,2,517,3586,154
sadness,392,455,125,7333


# Inspect results on test

In [73]:
predictions = model_3.predict(x_test_input)


winning_class = np.apply_along_axis(lambda x: np.argmax(x), 1, predictions)
winning_class_pred = np.apply_along_axis(lambda x: np.max(x), 1, predictions)
winning_class_name = label_encoder.inverse_transform(winning_class)

test_copie = X_test.copy()
test_copie["value"] = y_test.values
test_copie["pred"] = winning_class_name
test_copie["pred_prop"] = winning_class_pred

test_copie["match"] = test_copie.apply(lambda x: int(x.value==x.pred), axis=1)
test_copie.head()

,text,value,pred,pred_prop,match
9148,hey that link didn t work for me,joy,sadness,0.350658,0
6777,i know how you feel i was physically abused as...,sadness,sadness,0.999401,1
37594,barely awake eating breakfast marathon morning...,joy,joy,0.431956,1
16630,i have problems wih pizza ii love it too much ...,joy,love,0.414910,0
3330,i grinned at peter feeling somehow triumphant ...,joy,joy,0.999988,1


In [74]:
print(test_copie[test_copie.match==1].value.value_counts())
print(test_copie[test_copie.match==0].value.value_counts())
pd.crosstab(test_copie.value, test_copie.pred)

joy        1641
sadness    1476
love        778
anger       586
Name: value, dtype: int64
joy        1182
sadness     600
love        287
anger       188
Name: value, dtype: int64


pred,anger,joy,love,sadness
value,,,,
anger,586,33,36,119
joy,93,1641,753,336
love,42,129,778,116
sadness,237,189,174,1476


# Inspect Model on raw test

In [152]:
sentence_test = "i miss my husband"

test_clean = emb_processing.clean_text(pd.DataFrame({config.TEXT_COL:[sentence_test]}))
print(test_clean)
x_test, _ = emb_processing.tokenize_text(test_clean, istraining=False, tokenizer=tokenizer)

print(model_3.predict(x_test))

print(label_encoder.inverse_transform([0, 1, 2, 3]))

0    i miss my husband
Name: text, dtype: object
[[0.13728626 0.35766068 0.18658645 0.31846663]]
['anger' 'joy' 'love' 'sadness']


# Save model

In [157]:
from pathlib import Path

model_to_keep = model_3

# Create the directory for the model
pathlib.Path(config.LARGE_DATA_DIR).mkdir(parents=True, exist_ok=True)

label_dict = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))

pickle.dump(label_dict, open(config.LARGE_DATA_LABEL_DICT_PATH, "wb"))
pickle.dump(tokenizer, open(config.LARGE_DATA_TOKENIZER_PATH, "wb"))
tf.keras.models.save_model(
    model_to_keep, config.LARGE_DATA_MODEL_PATH, overwrite=True, include_optimizer=True, save_format="tf",
    signatures=None, options=None
)

INFO:tensorflow:Assets written to: /home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/../models/large_data/large_data_model/assets


# Try Bert

In [3]:
import tensorflow as tf
import transformers
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras as keras
from keras import Model

In [36]:
bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
bert_clf_model = transformers.TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

In [39]:
a = bert_clf_model.get_layer("bert")
b = bert_clf_model.layers[0]
print(a.)
print(b)

In [40]:
x_tokens[0]

array([ 101, 1045, 2514, 9202, 2087, 1997, 1996, 2051,  102,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0])

In [42]:
a(x_tokens)

ResourceExhaustedError: OOM when allocating tensor with shape[18000,70,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:AddV2]

In [33]:
model = tf.keras.Sequential([
    keras.layers.Input(shape=x_tokens[1:]),
    transformers.TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased').get_layer("distilbert"),
    # Adds a densely-connected layer with 64 units to the model:
    keras.layers.Dense(64, activation=tf.nn.relu),
    # Add another:
    keras.layers.Dense(64, activation=tf.nn.relu),
# Add an output layer with 10 output units:
    keras.layers.Dense(6, activation=tf.nn.softmax)])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(x=x_tokens,y=y_labels, verbose=2, epochs=3)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [31]:
x_tokens.shape

(18000, 70)

In [6]:
bert_clf_model.summary()
#bert_clf_model.get_layer("bert")

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  1538      
_________________________________________________________________
dropout_19 (Dropout)         multiple                  0         
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [7]:


data = data_initial.sample(frac=1).copy()
X = data.drop(config.TARGET_COL, axis=1)
y = data[config.TARGET_COL]

#x_clean = emb_processing.clean_text(X)
#x_input, tokenizer = emb_processing.tokenize_text(x_clean, istraining=True)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_labels = keras.utils.to_categorical(y_encoded)

In [26]:
def tokenize_with_bertTokenizer(dataframe):

    new_dataframe = pd.DataFrame(columns=["input_ids", "token_type_ids", "attention_mask"])
    

    #inputs_ids, token_type_ids, attention_masks = [], [], []
    tokens = []
    for index, row in enumerate(dataframe[config.TEXT_COL].values):
        
        tokens_dict = bert_tokenizer.encode_plus(
            row,
            max_length=config.MAX_LEN,
            pad_to_max_length=config.MAX_LEN,
        )
        #line = np.array([tokens_dict["input_ids"], tokens_dict["input_ids"], tokens_dict["input_ids"]])
        #tokens.append(line)
        
        tokens.append(tokens_dict["input_ids"])
    tokens = np.array(tokens)
        

    return(tokens)
#data[config.TEXT_COL].apply(lambda x: tokenize_with_bertTokenizer(x))
x_tokens  = tokenize_with_bertTokenizer(X)

In [9]:
#y_labels = y_labels.reshape((y_labels.shape[0], y_labels.shape[1], 1))
tf_x_dataset = tf.data.Dataset.from_tensor_slices((x_tokens, y_labels))
#tf_y_dataset = tf.data.Dataset.from_tensor_slices((y_labels))

In [27]:
b_model = engine_bert.BertNn(n_classes=6)
b_model.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
b_model.fit(x=x_tokens,y=y_labels, verbose=2, epochs=3)

Epoch 1/3
(<tf.Tensor 'bert_nn_9/distilbert/transformer/layer_._5/output_layer_norm/batchnorm/add_1:0' shape=(None, 70, 768) dtype=float32>,)


TypeError: in user code:

    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/engine/engine_bert.py:28 call  *
        x = self.pre_dense(x)
    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:897 __call__  **
        self._maybe_build(inputs)
    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:2416 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/keras/layers/core.py:1152 build
        input_shape = tensor_shape.TensorShape(input_shape)
    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorf

Epoch 1/3
Tensor("bert_nn_2/strided_slice:0", shape=(70,), dtype=int64)
Tensor("bert_nn_2/strided_slice_1:0", shape=(70,), dtype=int64)


StagingError: in user code:

    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/cedric/Workspace/Workspace_Python/kaggle/emotion-detection/flask-api/src/engine/engine_bert.py:24 call  *
        x = self.bert_layer([inputs_ids,attention_masks])
    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/transformers/modeling_tf_distilbert.py:457 call  *
        embedding_output = self.embeddings(input_ids, inputs_embeds=inputs_embeds)  # (bs, seq_length, dim)
    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/transformers/modeling_tf_distilbert.py:114 call  *
        return self._embedding(inputs, inputs_embeds=inputs_embeds, training=training)
    /home/cedric/CLUSTER/anaconda3/envs/my_env/lib/python3.6/site-packages/transformers/modeling_tf_distilbert.py:139 _embedding  *
        seq_length = shape_lis

,text
1878,i am under pressure at the place i spend most ...
10417,i feel lame for pretty much only using my phon...
1551,i normally associate with a tough workout movi...
5891,i also wouldnt mind a canon d mark iii if anyo...
5268,ive gotten so used to them to the extent that ...
...,...
5134,i hope to always remain grateful even when fee...
12419,i am inferior to them then i feel as i did as ...
8943,i feel ugly to stop being lazy so i dont embar...
11749,i feel so unimportant to all of them they all ...


In [19]:
bert_tokenizer.encode_plus(X[config.TEXT_COL].tolist()[0])

{'input_ids': [101, 1045, 2572, 2104, 3778, 2012, 1996, 2173, 1045, 5247, 2087, 1997, 2026, 2733, 2006, 2627, 3325, 1045, 2097, 7166, 2000, 2514, 2062, 12511, 2005, 2936, 6993, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [250]:
filtre_file = "sadness"
temp = pd.read_csv("../inputs/train.txt",header=None, sep=";", names=["text", "sentiment"])
temp = temp[temp.sentiment==filtre_file]

print(temp.shape)
temp["taille"] = temp.text.apply(lambda x: len(x.split()))
temp = temp[temp.taille<30]
print(temp.head(n=30))
print(temp.shape)
temp = temp[["text", "sentiment"]]
temp.to_csv(f"./{filtre_file}.csv", index=False)

(4666, 2)
                                                  text sentiment  taille
0                              i didnt feel humiliated   sadness       4
1    i can go from feeling so hopeless to so damned...   sadness      21
5    ive been feeling a little burdened lately wasn...   sadness      12
10   i feel like i have to make the suffering i m s...   sadness      14
13                  i feel low energy i m just thirsty   sadness       8
16                i didnt really feel that embarrassed   sadness       6
17             i feel pretty pathetic most of the time   sadness       8
18   i started feeling sentimental about dolls i ha...   sadness      23
26   i feel so inhibited in someone elses kitchen l...   sadness      15
27              i become overwhelmed and feel defeated   sadness       6
45   i was feeling listless from the need of new th...   sadness      12
51   i found myself feeling a little discouraged th...   sadness       9
55   i feel you know basically like a fak